In [2]:
# !pip install -q kaggle
# !pip install datawig
!pip install catboost
# !pip install featexp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [3]:
# from google.colab import files
# files.upload()

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
!cp "/content/drive/MyDrive/Colab Notebooks/kaggle.json" "/content/kaggle.json"
!mkdir ~/.kaggle/
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c spaceship-titanic

  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 87.8MB/s]


In [6]:
!unzip spaceship-titanic.zip

Archive:  spaceship-titanic.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# Загрузка библиотек

In [87]:
import math
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(8, 4)})
sns.color_palette("pastel")
import warnings
warnings.filterwarnings('ignore')
import missingno as msno
# from featexp import get_univariate_plots
# from featexp import get_trend_stats
from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [169]:
PATH = '/content/'
train_dataframe = pd.read_csv('/content/train.csv')
test_dataframe = pd.read_csv('/content/test.csv')
submission = pd.read_csv("/content/sample_submission.csv")
df = pd.concat([train_dataframe, test_dataframe], axis=0).reset_index(drop=True)
print('train_dataframe', train_dataframe.shape)
print('test_dataframe', test_dataframe.shape)
print('df', df.shape)

train_dataframe (8693, 14)
test_dataframe (4277, 13)
df (12970, 14)


# Feature engineering

In [170]:
def feature_engineering(df):
    def fix_5500(df):
        if df > 5500:
            return 5500
        else:
            return df
    def fix_12500(df):
        if df > 12500:
            return 12500
        else:
            return df
    def fix_22000(df):
        if df > 22000:
            return 22000
        else:
            return df

    ### PassengerId
    df['id_group'] = df['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)
    df['passenger_num'] = df['PassengerId'].apply(lambda x: x.split('_')[1]).astype(int)
    df = pd.merge(df, df[['id_group', 'PassengerId']].groupby('id_group').count().reset_index().rename(columns={'PassengerId': 'group_size'}), how='left', on='id_group')
    df = df.drop('PassengerId', axis=1)
    df = df.drop('id_group', axis=1)
  
    ### Name
    df['Name'] = df['Name'].fillna('Unknown Unknown')
    # df['Name'] = df['Name'].fillna(method='ffill')
    def last_name(x):
        try:
            if x == type(float):
                return np.NaN
            else:
                return x.split(" ")[1]
        except:
            return np.NaN  
    df['last_name'] = df['Name'].apply(last_name).astype(str)
    df['last_name_len'] = df['last_name'].map(lambda x: len(x))
    df['name_len'] = df['Name'].map(lambda x: len(x))
    df = df.drop('Name', axis=1)
    df = df.drop('last_name', axis=1)
  
    ### Total_bill
    def to_bool_5500(df):
        if df == 5500:
            return 1
        else:
            return 0
    def to_bool_12500(df):
        if df == 12500:
            return 1
        else:
            return 0
    df['RoomService'] = df['RoomService'].fillna(0).apply(fix_5500)
    df['RoomService_bool'] = df['RoomService'].apply(to_bool_5500)

    df['FoodCourt'] = df['FoodCourt'].fillna(0).apply(fix_12500)
    df['FoodCourt_bool'] = df['FoodCourt'].apply(to_bool_12500)

    df['ShoppingMall'] = df['ShoppingMall'].fillna(0).apply(fix_5500)
    df['ShoppingMall_bool'] = df['ShoppingMall'].apply(to_bool_5500)

    df['Spa'] = df['Spa'].fillna(0).apply(fix_12500)
    df['Spa_bool'] = df['Spa'].apply(to_bool_12500)

    df['VRDeck'] = df['VRDeck'].fillna(0).apply(fix_12500)
    df['VRDeck_bool'] = df['VRDeck'].apply(to_bool_12500)

    def to_bool_22000(df):
        if df == 22000:
            return 1
        else:
            return 0
    df['total'] = (df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck']).apply(fix_22000)
    df['total_bool'] = df['total'].apply(to_bool_22000)

    def paid_to_family(df):
        if df == 22000:
            return 1
        else:
            return 0
    df['paid_to_family'] = df['total'].apply(paid_to_family)


    ### Cabin
    # df['Cabin'] = df['Cabin'].fillna('Unknown/9999/Unknown')
    df['Cabin'] = df['Cabin'].fillna('Unknown/2000/Unknown')
    df['Cabin_deck'] = df['Cabin'].apply(lambda x: x.split('/')[0]).astype(str)
    df['Cabin_num'] = df['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
    df['Cabin_deck_group'] = pd.cut(x=df['Cabin_num'], bins=[-1, 3, 107, 295, 342, 343, 354, 609, 1508], labels=['1', '2', '3', '4', '5', '6', '7', '8']).astype(str)
    df['Cabin_side'] = df['Cabin'].apply(lambda x: x.split('/')[2]).astype(str)
    df['Cabin_len'] = df['Cabin'].apply(lambda x: len(x)).astype(int)
    df = df.drop('Cabin', axis=1)

    ### Age
    df['Age'] = df['Age'].fillna(df['Age'].mean()-5)
    df['Age_group'] = pd.cut(x=df['Age'], bins=[-1, 5, 10, 15, 20, 30, 40, 50, 60, 70, 75, 100], labels=['0-5', '6-10', '11-15', '16-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-75', '76-100']).astype(str)

    ### HomePlanet
    df['HomePlanet'] = df['HomePlanet'].fillna(method='ffill')

    ### CryoSleep
    df['CryoSleep'] = df['CryoSleep'].fillna(method='ffill').astype(int)

    ### Destination
    df['Destination'] = df['Destination'].fillna(method='ffill')
  
    ### VIP
    df['VIP'] = df['VIP'].fillna(method='ffill').astype(int)

    return df

In [171]:
df = feature_engineering(df)

In [172]:
train_dataframe = df[df['Transported'].notnull()]
test_dataframe = df[df['Transported'].isnull()]
print('train_dataframe', train_dataframe.shape)
print('test_dataframe', test_dataframe.shape)

train_dataframe (8693, 29)
test_dataframe (4277, 29)


In [173]:
feats_cat = []
feats_num = []
for col in train_dataframe.columns:
    if col in ['PassengerId', 'Transported']:
        continue
    if train_dataframe[col].dtype == 'object':
        feats_cat += [col]
    else:
        feats_num += [col]
        
X_cat = pd.get_dummies(train_dataframe[feats_cat])
X_num = train_dataframe[feats_num]
X_pred_cat = pd.get_dummies(test_dataframe[feats_cat])
X_pred_num = test_dataframe[feats_num]

X = pd.concat([X_cat, X_num], axis=1)
y = train_dataframe['Transported'].astype(int)
X_pred = pd.concat([X_pred_cat, X_pred_num], axis=1)

print('X', X.shape)
print('y', y.shape)
print('X_pred', X_pred.shape)

X (8693, 60)
y (8693,)
X_pred (4277, 60)


In [177]:
col_names = list(X.columns)
s_scaler = StandardScaler()
X_scaled= s_scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=col_names)   

X_pred_scaled= s_scaler.fit_transform(X_pred)
X_pred_scaled = pd.DataFrame(X_pred_scaled, columns=col_names)   

In [180]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Catboost

In [189]:
clf=CatBoostClassifier(
                         iterations=10000,
                         learning_rate=0.023,
                         max_depth=6,
                         l2_leaf_reg=22,
                         verbose=1000,
                         random_seed=42,
                         eval_metric='Accuracy',
                         early_stopping_rounds=500,
                        )
# clf.fit(X_scaled, y)
# clf.fit(X,y,eval_set=(X,y),use_best_model=True)
clf.fit(X_train,y_train,eval_set=(X_test,y_test),use_best_model=True)
# clf.fit(X_train, y_train)
clf.score(X_test, y_test)


0:	learn: 0.7506868	test: 0.7434646	best: 0.7434646 (0)	total: 13.1ms	remaining: 2m 11s
1000:	learn: 0.8683036	test: 0.8062043	best: 0.8069014 (986)	total: 6.99s	remaining: 1m 2s
2000:	learn: 0.9038462	test: 0.8138724	best: 0.8142210 (1909)	total: 12.2s	remaining: 48.7s
3000:	learn: 0.9273695	test: 0.8128268	best: 0.8163123 (2552)	total: 17.3s	remaining: 40.4s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8163123039
bestIteration = 2552

Shrink model to first 2553 iterations.


0.8163123039386546

# Features checking

# Predict

In [108]:
predictions = clf.predict(X_pred)#
predictions = predictions.astype(str)
predictions = np.char.replace(predictions, '0', 'False')
predictions = np.char.replace(predictions, '1', 'True')

In [109]:
submission['Transported'] = predictions
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")

In [110]:
!kaggle competitions submit -c spaceship-titanic -f submission.csv -m "train_test_split clear"

100% 56.3k/56.3k [00:02<00:00, 21.3kB/s]
400 - Bad Request
